## 1. Configuração Inicial e Instalação de Dependências

In [21]:
!pip install prophet -q
!pip install pandas-gbq google-cloud-bigquery pyarrow -q

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\mathe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\prophet\\stan_model\\cmdstan-2.37.0\\stan\\lib\\stan_math\\lib\\tbb_2020.3\\include\\tbb\\internal\\_deprecated_header_message_guard.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\mathe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\mathe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.

## 2. Importação de Bibliotecas

In [25]:
import os
import pandas as pd
import numpy as np
from prophet import Prophet
from google.cloud import bigquery
from pandas_gbq import to_gbq
from dotenv import load_dotenv
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [26]:
# Célula de Inicialização de Variáveis e Credenciais

# 1. Carrega todas as variáveis do arquivo .env para o ambiente
load_dotenv(r"C:\Users\mathe\Downloads\archive (10)\.env.txt")

# 2. Garante que o caminho da chave de serviço (ADC)

chave_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if chave_path:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = chave_path
    print("Caminho da chave de serviço configurado via os.environ.")
else:
    print("Atenção: GOOGLE_APPLICATION_CREDENTIALS não foi encontrado no .env!")

Caminho da chave de serviço configurado via os.environ.


## 2. Extração de Dados (BigQuery)

In [24]:
client = bigquery.Client(project="challenge-421602")  
query = """
    SELECT 
      DATA,
      ANO,
      MES,
      LOJA,
      CIDADE,
      TINTA,
      COR_TIPO,
      QUANTIDADE_VENDIDA,
      VALOR_TOTAL_VENDIDO,
      VISITANTES_TINTAS,
      VISITANTES_LOJA,
      PROMOCAO,
      INCC,
      PMCC,
      ALVARA,
      CENSO,
      IAC,
      DESEMPREGO,
      INFLACAO,
      TEMPERATURA,
      INDICE_PLUVIOMETRICO
    FROM `challenge-421602.vendas.vendas_sp_previsao`
    ORDER BY DATA
"""
df = client.query(query).to_dataframe()
print(df.head())

C:\Users\mathe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


         DATA   ANO  MES                        LOJA     CIDADE  \
0  2023-01-01  2023    1  São Paulo - Marginal Tietê  São Paulo   
1  2023-01-01  2023    1  São Paulo - Marginal Tietê  São Paulo   
2  2023-01-01  2023    1          São Paulo - Centro  São Paulo   
3  2023-01-01  2023    1  São Paulo - Marginal Tietê  São Paulo   
4  2023-01-01  2023    1          São Paulo - Centro  São Paulo   

               TINTA     COR_TIPO  QUANTIDADE_VENDIDA  VALOR_TOTAL_VENDIDO  \
0       Tinta Branca       Branca                 122         23322.868133   
1    Tinta Anti-Mofo    Anti-Mofo                 118         23167.579501   
2    Tinta Brilhante    Brilhante                 152         11736.794327   
3  Tinta Impermeável  Impermeável                  87         12229.004627   
4    Tinta Anti-Mofo    Anti-Mofo                 138         26907.533593   

   VISITANTES_TINTAS  ...  PROMOCAO        INCC       PMCC     ALVARA  \
0                839  ...         0  117.446660  54.881

### 2.1. Análise Exploratória Rápida

In [9]:
# Conferir lojas e produtos
print("Lojas:", df['LOJA'].unique())
print("Produtos:", df['TINTA'].unique())


Lojas: ['São Paulo - Marginal Tietê' 'São Paulo - Centro']
Produtos: ['Tinta Branca' 'Tinta Anti-Mofo' 'Tinta Brilhante' 'Tinta Impermeável'
 'Tinta Esmalte' 'Tinta Spray' 'Tinta Preta' 'Tinta Acrílica'
 'Tinta Perolada' 'Tinta Azul' 'Tinta Fosca' 'Tinta Metálica'
 'Tinta Vermelha' 'Tinta Amarela']


## 3. Pré-Processamento dos dados

In [ ]:
# Definir os feriados
feriados_dict = {
    'Carnaval': ['2023-02-20', '2024-02-12'],
    'Páscoa': ['2023-04-09', '2024-03-31'],
    'Natal': ['2023-12-25', '2024-12-25'],
    '7 de Setembro': ['2023-09-07', '2024-09-07']
}

feriados_lista = []
for nome, datas in feriados_dict.items():
    for data in datas:
        feriados_lista.append({'holiday': nome, 'ds': pd.to_datetime(data)})

feriados_df = pd.DataFrame(feriados_lista)


In [ ]:
# Preparação
df['DATA'] = pd.to_datetime(df['DATA'])

# Variáveis exógenas que vamos usar
exog_vars = [
    'PROMOCAO', 'VISITANTES_TINTAS', 'VISITANTES_LOJA', 'INCC',
    'PMCC', 'ALVARA', 'CENSO', 'IAC', 'DESEMPREGO', 'INFLACAO',
    'TEMPERATURA', 'INDICE_PLUVIOMETRICO'
]

## 4. Modelagem e Previsão

In [ ]:
resultados_previsao = []
resultados_metricas = []

lojas = df['LOJA'].unique()
tintas = df['TINTA'].unique()

for loja in lojas:
    for tinta in tintas:
        subset = df[(df['LOJA'] == loja) & (df['TINTA'] == tinta)].copy()
        if subset.shape[0] < 12:
            continue

        subset = subset.sort_values('DATA')

        ultima_data_disponivel = subset['DATA'].max()
        mes_teste = pd.to_datetime(ultima_data_disponivel).to_period("M")

        treino = subset[subset['DATA'].dt.to_period("M") < mes_teste]
        teste = subset[subset['DATA'].dt.to_period("M") == mes_teste]

        if treino.shape[0] < 12 or teste.empty:
            continue

        treino_prophet = treino[['DATA', 'QUANTIDADE_VENDIDA'] + exog_vars].rename(columns={'DATA': 'ds', 'QUANTIDADE_VENDIDA': 'y'})
        teste_prophet = teste[['DATA', 'QUANTIDADE_VENDIDA'] + exog_vars].rename(columns={'DATA': 'ds', 'QUANTIDADE_VENDIDA': 'y'})

        # Forçar todos os exógenos como float64
        for col in treino_prophet.columns:
            if col != 'ds':
                treino_prophet[col] = pd.to_numeric(treino_prophet[col], errors='coerce').astype('float64')
        for col in teste_prophet.columns:
            if col != 'ds':
                teste_prophet[col] = pd.to_numeric(teste_prophet[col], errors='coerce').astype('float64')

        # Diagnóstico: tipos das colunas
        print(f"Tipos em teste_prophet ({loja} - {tinta}):\n{teste_prophet.dtypes}")

        # Verificação de colunas com tipo inválido
        colunas_invalidas = teste_prophet.drop(columns='ds').select_dtypes(exclude=['number']).columns
        if not colunas_invalidas.empty:
            print(f"Colunas com tipo inválido em {loja} - {tinta}: {list(colunas_invalidas)}")
            print(teste_prophet[colunas_invalidas].head())
            continue

        # Verificação de valores nulos
        if teste_prophet.drop(columns='ds').isnull().any().any():
            print(f"Valores nulos em teste_prophet para {loja} - {tinta}. Pulando.")
            print(teste_prophet[teste_prophet.isnull().any(axis=1)])
            continue

        modelo = Prophet(
            holidays=feriados_df,
            changepoint_prior_scale=0.2,
            seasonality_mode='multiplicative',
            seasonality_prior_scale=10
        )

        modelo.add_regressor('PROMOCAO')
        for var in exog_vars:
            if var != 'PROMOCAO':
                modelo.add_regressor(var)

        modelo.fit(treino_prophet)

        forecast = modelo.predict(teste_prophet)

        y_true = teste_prophet['y'].values
        y_pred = forecast['yhat'].values

        mae = mean_absolute_error(y_true, y_pred)
        rmse = mean_squared_error(y_true, y_pred) ** 0.5


        resultados_metricas.append({
            'LOJA': loja,
            'TINTA': tinta,
            'MAE': mae,
            'RMSE': rmse
        })

        for i in range(len(teste_prophet)):
            resultados_previsao.append({
                'LOJA': loja,
                'TINTA': tinta,
                'DATA': teste_prophet['ds'].iloc[i],
                'VENDA_REAL': y_true[i],
                'VENDA_PREVISTA': y_pred[i]
            })


📊 Tipos em teste_prophet (São Paulo - Marginal Tietê - Tinta Branca):
ds                      datetime64[ns]
y                              float64
PROMOCAO                       float64
VISITANTES_TINTAS              float64
VISITANTES_LOJA                float64
INCC                           float64
PMCC                           float64
ALVARA                         float64
CENSO                          float64
IAC                            float64
DESEMPREGO                     float64
INFLACAO                       float64
TEMPERATURA                    float64
INDICE_PLUVIOMETRICO           float64
dtype: object


17:38:33 - cmdstanpy - INFO - Chain [1] start processing
17:38:33 - cmdstanpy - INFO - Chain [1] done processing
17:38:33 - cmdstanpy - INFO - Chain [1] start processing
17:38:34 - cmdstanpy - INFO - Chain [1] done processing


📊 Tipos em teste_prophet (São Paulo - Marginal Tietê - Tinta Anti-Mofo):
ds                      datetime64[ns]
y                              float64
PROMOCAO                       float64
VISITANTES_TINTAS              float64
VISITANTES_LOJA                float64
INCC                           float64
PMCC                           float64
ALVARA                         float64
CENSO                          float64
IAC                            float64
DESEMPREGO                     float64
INFLACAO                       float64
TEMPERATURA                    float64
INDICE_PLUVIOMETRICO           float64
dtype: object


17:38:34 - cmdstanpy - INFO - Chain [1] start processing
17:38:34 - cmdstanpy - INFO - Chain [1] done processing


📊 Tipos em teste_prophet (São Paulo - Marginal Tietê - Tinta Brilhante):
ds                      datetime64[ns]
y                              float64
PROMOCAO                       float64
VISITANTES_TINTAS              float64
VISITANTES_LOJA                float64
INCC                           float64
PMCC                           float64
ALVARA                         float64
CENSO                          float64
IAC                            float64
DESEMPREGO                     float64
INFLACAO                       float64
TEMPERATURA                    float64
INDICE_PLUVIOMETRICO           float64
dtype: object


...

17:38:38 - cmdstanpy - INFO - Chain [1] start processing
17:38:38 - cmdstanpy - INFO - Chain [1] done processing
17:38:38 - cmdstanpy - INFO - Chain [1] start processing


📊 Tipos em teste_prophet (São Paulo - Centro - Tinta Fosca):
ds                      datetime64[ns]
y                              float64
PROMOCAO                       float64
VISITANTES_TINTAS              float64
VISITANTES_LOJA                float64
INCC                           float64
PMCC                           float64
ALVARA                         float64
CENSO                          float64
IAC                            float64
DESEMPREGO                     float64
INFLACAO                       float64
TEMPERATURA                    float64
INDICE_PLUVIOMETRICO           float64
dtype: object


17:38:38 - cmdstanpy - INFO - Chain [1] done processing
17:38:38 - cmdstanpy - INFO - Chain [1] start processing
17:38:38 - cmdstanpy - INFO - Chain [1] done processing


📊 Tipos em teste_prophet (São Paulo - Centro - Tinta Metálica):
ds                      datetime64[ns]
y                              float64
PROMOCAO                       float64
VISITANTES_TINTAS              float64
VISITANTES_LOJA                float64
INCC                           float64
PMCC                           float64
ALVARA                         float64
CENSO                          float64
IAC                            float64
DESEMPREGO                     float64
INFLACAO                       float64
TEMPERATURA                    float64
INDICE_PLUVIOMETRICO           float64
dtype: object
📊 Tipos em teste_prophet (São Paulo - Centro - Tinta Amarela):
ds                      datetime64[ns]
y                              float64
PROMOCAO                       float64
VISITANTES_TINTAS              float64
VISITANTES_LOJA                float64
INCC                           float64
PMCC                           float64
ALVARA                         float64
C

17:38:39 - cmdstanpy - INFO - Chain [1] start processing
17:38:39 - cmdstanpy - INFO - Chain [1] done processing


## 5. Consolidação e insert de dados

In [ ]:
# Avaliação
previsoes_df = pd.DataFrame(resultados_previsao)
metricas_df = pd.DataFrame(resultados_metricas)

### 5.1. Visualização dos Resultados

In [14]:
metricas_df

,LOJA,TINTA,MAE,RMSE
0,São Paulo - Marginal Tietê,Tinta Branca,10.391252,10.391252
1,São Paulo - Marginal Tietê,Tinta Anti-Mofo,18.704733,18.704733
2,São Paulo - Marginal Tietê,Tinta Brilhante,59.363242,59.363242
3,São Paulo - Marginal Tietê,Tinta Impermeável,9.460917,9.460917
4,São Paulo - Marginal Tietê,Tinta Esmalte,23.142424,23.142424
5,São Paulo - Marginal Tietê,Tinta Spray,76.635710,76.635710
6,São Paulo - Marginal Tietê,Tinta Preta,11.822945,11.822945
7,São Paulo - Marginal Tietê,Tinta Acrílica,32.663352,32.663352
8,São Paulo - Marginal Tietê,Tinta Perolada,14.333442,14.333442
9,São Paulo - Marginal Tietê,Tinta Azul,17.318362,17.318362


In [15]:
previsoes_df

,LOJA,TINTA,DATA,VENDA_REAL,VENDA_PREVISTA
0,São Paulo - Marginal Tietê,Tinta Branca,2024-12-01,171.0,160.608748
1,São Paulo - Marginal Tietê,Tinta Anti-Mofo,2024-12-01,127.0,108.295267
2,São Paulo - Marginal Tietê,Tinta Brilhante,2024-12-01,259.0,199.636758
3,São Paulo - Marginal Tietê,Tinta Impermeável,2024-12-01,133.0,123.539083
4,São Paulo - Marginal Tietê,Tinta Esmalte,2024-12-01,318.0,341.142424
5,São Paulo - Marginal Tietê,Tinta Spray,2024-12-01,336.0,259.364290
6,São Paulo - Marginal Tietê,Tinta Preta,2024-12-01,132.0,143.822945
7,São Paulo - Marginal Tietê,Tinta Acrílica,2024-12-01,183.0,150.336648
8,São Paulo - Marginal Tietê,Tinta Perolada,2024-12-01,124.0,109.666558
9,São Paulo - Marginal Tietê,Tinta Azul,2024-12-01,187.0,169.681638


In [16]:
resultados_metricas = []
resultados_previsao_futuro = []

for loja in df['LOJA'].unique():
    for tinta in df['TINTA'].unique():
        subset = df[(df['LOJA'] == loja) & (df['TINTA'] == tinta)].copy()
        if subset.shape[0] < 12:
            continue

        subset = subset.sort_values('DATA')

        ultima_data = subset['DATA'].max()
        mes_teste = ultima_data.to_period("M")

        treino = subset[subset['DATA'].dt.to_period("M") < mes_teste]
        teste = subset[subset['DATA'].dt.to_period("M") == mes_teste]

        if treino.shape[0] < 12 or teste.empty:
            continue

        treino_prophet = treino[['DATA', 'QUANTIDADE_VENDIDA'] + exog_vars].rename(columns={'DATA': 'ds', 'QUANTIDADE_VENDIDA': 'y'})
        teste_prophet = teste[['DATA', 'QUANTIDADE_VENDIDA'] + exog_vars].rename(columns={'DATA': 'ds', 'QUANTIDADE_VENDIDA': 'y'})

        # Tipagem
        for col in treino_prophet.columns:
            if col != 'ds':
                treino_prophet[col] = pd.to_numeric(treino_prophet[col], errors='coerce').astype('float64')
        for col in teste_prophet.columns:
            if col != 'ds':
                teste_prophet[col] = pd.to_numeric(teste_prophet[col], errors='coerce').astype('float64')

        if treino_prophet[exog_vars].isnull().any().any() or teste_prophet[exog_vars].isnull().any().any():
            continue

        modelo = Prophet(
            holidays=feriados_df,
            changepoint_prior_scale=0.2,
            seasonality_mode='multiplicative',
            seasonality_prior_scale=10
        )

        modelo.add_regressor('PROMOCAO')
        for var in exog_vars:
            if var != 'PROMOCAO':
                modelo.add_regressor(var)

        modelo.fit(treino_prophet)

        # --- MÉTRICAS DO MÊS REAL ---
        forecast_teste = modelo.predict(teste_prophet)
        y_true = teste_prophet['y'].values
        y_pred = forecast_teste['yhat'].values

        mae = mean_absolute_error(y_true, y_pred)
        rmse = mean_squared_error(y_true, y_pred) ** 0.5

        resultados_metricas.append({
            'LOJA': loja,
            'TINTA': tinta,
            'MES': mes_teste.strftime('%Y-%m'),
            'MAE': mae,
            'RMSE': rmse
        })

        # --- PREVISÃO PARA MÊS FUTURO ---
        proxima_data = pd.date_range(start=ultima_data + pd.offsets.MonthBegin(1), periods=1, freq='MS')
        df_futuro = pd.DataFrame({'ds': proxima_data})

        for var in exog_vars:
            df_futuro[var] = subset[var].mean()

        for col in exog_vars:
            df_futuro[col] = pd.to_numeric(df_futuro[col], errors='coerce').astype('float64')

        if df_futuro[exog_vars].isnull().any().any():
            continue

        forecast_futuro = modelo.predict(df_futuro)

        resultados_previsao_futuro.append({
            'LOJA': loja,
            'TINTA': tinta,
            'DATA': df_futuro['ds'].iloc[0],
            'VENDA_PREVISTA': forecast_futuro['yhat'].iloc[0]
        })


17:39:05 - cmdstanpy - INFO - Chain [1] start processing
17:39:05 - cmdstanpy - INFO - Chain [1] done processing
17:39:05 - cmdstanpy - INFO - Chain [1] start processing
17:39:05 - cmdstanpy - INFO - Chain [1] done processing
17:39:05 - cmdstanpy - INFO - Chain [1] start processing
17:39:05 - cmdstanpy - INFO - Chain [1] done processing
17:39:06 - cmdstanpy - INFO - Chain [1] start processing
17:39:06 - cmdstanpy - INFO - Chain [1] done processing
17:39:06 - cmdstanpy - INFO - Chain [1] start processing
17:39:06 - cmdstanpy - INFO - Chain [1] done processing
17:39:06 - cmdstanpy - INFO - Chain [1] start processing
17:39:06 - cmdstanpy - INFO - Chain [1] done processing
17:39:07 - cmdstanpy - INFO - Chain [1] start processing
17:39:07 - cmdstanpy - INFO - Chain [1] done processing
17:39:07 - cmdstanpy - INFO - Chain [1] start processing
17:39:07 - cmdstanpy - INFO - Chain [1] done processing
17:39:07 - cmdstanpy - INFO - Chain [1] start processing
17:39:07 - cmdstanpy - INFO - Chain [1]

In [17]:
df_previsoes = pd.DataFrame(resultados_previsao_futuro)
df_previsoes['VENDA_PREVISTA'] = df_previsoes['VENDA_PREVISTA'].astype('int')

### 5.2. Inserção de dados para o BigQuery

In [ ]:
# Defina o seu ID do projeto
project_id = 'challenge-421602'

# Tabela de métricas
to_gbq(
    metricas_df,
    destination_table='vendas.metricas_modelo',
    project_id=project_id,
    if_exists='append'  # ou 'replace' para sobrescrever
)

# Tabela de previsões futuras
to_gbq(
    df_previsoes,
    destination_table='vendas.previsoes',
    project_id=project_id,
    if_exists='append'
)


100%|██████████| 1/1 [00:00<00:00, 1004.14it/s]
